# g) Ngonye Falls Climate Change Impact Stress Test



## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|



## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|



## Libraries and Setup

In [2]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [3]:
input_data='./input_data/'
output_data='./output_data/'

## Load Data

Daily data

In [47]:
daily=pd.read_csv(output_data + "ngonye_flow_daily.csv").set_index('Date')
daily.tail(5)

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,WaterMonth,WaterDay,WaterWeek,Volume
Date,,,,,,,,,,,,,,
2017-09-26,2017-10-07,204.0,0.995184,203.017496,0.950,2017,9,26,2017.09,2016,12,361,52,0.017541
2017-09-27,2017-10-08,204.0,0.995184,203.017496,0.950,2017,9,27,2017.09,2016,12,362,52,0.017541
2017-09-28,2017-10-09,204.0,0.995184,203.017496,0.950,2017,9,28,2017.09,2016,12,363,52,0.017541
2017-09-29,2017-10-10,204.0,0.995184,203.017496,0.950,2017,9,29,2017.09,2016,12,364,52,0.017541
2017-09-30,2017-10-11,201.0,0.986217,198.229576,0.958,2017,9,30,2017.09,2016,12,365,53,0.017127


Load the 'worst case scenario' climate change impact flow change percentages. These are by month for WC1 and WC2.

In [10]:
wcs=pd.read_csv(input_data + "climate_change_worst_cases.csv").set_index('Month')
wcs

,WC1,WC2
Month,,
10,-0.16,-0.24
11,-0.20,-0.30
12,-0.17,-0.40
1,-0.18,-0.33
2,-0.16,-0.32
3,-0.41,-0.40
4,-0.29,-0.39
5,-0.24,-0.36
6,-0.24,-0.35


DatetimeIndex(['2017-01-10', '2017-01-11', '2017-01-12', '2017-01-13',
               '2017-01-14', '2017-01-15', '2017-01-16', '2017-01-17',
               '2017-01-18', '2017-01-19',
               ...
               '2053-09-21', '2053-09-22', '2053-09-23', '2053-09-24',
               '2053-09-25', '2053-09-26', '2053-09-27', '2053-09-28',
               '2053-09-29', '2053-09-30'],
              dtype='datetime64[ns]', length=13413, freq='D')

In [39]:
wc1=pd.DataFrame(index=pd.date_range(start='10/1/2017', end='30/09/2053'))
wc1.index.rename('Date',inplace=True)
wc1['SourceDate']=wc1.index-pd.DateOffset(years=93)  #pd.date_range(start='1/10/1924',freq='D',periods=len(wc1.index))
wc1

,SourceDate
Date,
2017-10-01,1924-10-01
2017-10-02,1924-10-02
2017-10-03,1924-10-03
2017-10-04,1924-10-04
2017-10-05,1924-10-05
...,...
2053-09-26,1960-09-26
2053-09-27,1960-09-27
2053-09-28,1960-09-28


In [48]:
wc1['Year']=wc1.index.year
wc1['Month']=wc1.index.month
wc1['WaterYear']=wc1.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
wc1['YearsSince1990']=wc1['Year']-1990
wc1

,SourceDate,Year,Month,WaterYear,YearsSince1990,Factor
Date,,,,,,
2017-10-01,1924-10-01,2017,10,2017,27,-0.05400
2017-10-02,1924-10-02,2017,10,2017,27,-0.05400
2017-10-03,1924-10-03,2017,10,2017,27,-0.05400
2017-10-04,1924-10-04,2017,10,2017,27,-0.05400
2017-10-05,1924-10-05,2017,10,2017,27,-0.05400
...,...,...,...,...,...,...
2053-09-26,1960-09-26,2053,9,2052,63,-0.11025
2053-09-27,1960-09-27,2053,9,2052,63,-0.11025
2053-09-28,1960-09-28,2053,9,2052,63,-0.11025


In [49]:
wc1['Factor']=wc1.join(wcs,on='Month')['WC1']/80*wc1['YearsSince1990']
wc1.join(daily,on='SourceDate')['Flow']

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat